In [ ]:
import livecell_tracker.sample_data
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from napari.layers import Shapes


In [ ]:
dic_dataset_path = Path("../datasets/test_data_STAV-A549/DIC_data")
mask_dataset_path = Path("../datasets/test_data_STAV-A549/mask_data")
dic_dataset, mask_dataset = livecell_tracker.sample_data.tutorial_three_image_sys(dic_dataset_path, mask_dataset_path)


In [ ]:
from livecell_tracker.core.io_sc import prep_scs_from_mask_dataset
single_cells = prep_scs_from_mask_dataset(mask_dataset, dic_dataset)

In [ ]:
from livecell_tracker.core import SingleCellTrajectory, SingleCellStatic

class ScSegOperator:
    def __init__(
        self,
        sc: SingleCellStatic,
        viewer,
        shape_layer: Shapes=None,
        face_color=(0, 0, 1, 1),
    ):
        self.sc = sc
        self.viewer = viewer
        self.shape_layer = shape_layer
        self.face_color = face_color

    def create_sc_layer(self, name=None, contour_sample_num=100):
        if name is None:
            name = f"sc_{self.sc.id}"
        shape_vec = self.sc.get_napari_shape_contour_vec(contour_sample_num=contour_sample_num)
        properties = {"sc": [self.sc]}
        print("shape vec", shape_vec)
        shape_layer = self.viewer.add_shapes(
            [shape_vec],
            properties=properties,
            face_color=[self.face_color],
            shape_type="polygon",
            name=name,
        )
        self.shape_layer = shape_layer



In [ ]:
import napari
viewer = napari.view_image(dic_dataset.to_dask(), name="dic_image", cache=True)

In [ ]:
sample_sc = single_cells[0]
sample_sc_seg_operator = ScSegOperator(sample_sc, viewer=viewer)
sample_sc_seg_operator.create_sc_layer(contour_sample_num=20)